# ESFS example workflow - Delile et al. 2019 mouse neural tube development scRNA-sequencing data
In this workflow we apply ESFS to a mouse embryo neural tube developmental dataset from Delile et al. 2019. As demonstrated in the UMAP below and detailed in our manuscript, ESFS allow us to identify high resultion developmental trajectories and cell states that failed to be identified by conventional scRNA-seq workflows such as Scanpy and Seurat.

In this workflow ESFS is used to perform unsupervised feature selection to subset the mouse neural tube dataset from 19179 down to 879 genes. Subsetting down to the 879 genes identified by ESFS produces a UMAP displaying biologically relevent trajactories and cell types without needing apply PCA or data smoothing/imputation, indicating that ESFS has identified a set of highly biologically informative genes, which in turn facilitates improved downstream analysis.

In [ ]:
### Data path
path = "/Users/radleya/The Francis Crick Dropbox/BriscoeJ/Radleya/New_ES_Packages/Delile_Data/"
from IPython.display import Image, display
display(Image(filename=path+"Delile_2019_ESFS_UMAP.png"))

### Import ESFS package

In [ ]:
import ESFS
from scipy.sparse import csc_matrix

### Set python default discrete class colour palette to one with more colours

In [ ]:
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import anndata as ad
Colours = px.colors.qualitative.Dark24
Colours.remove('#222A2A') # Remove black form the color palette (personal preference).
#Colours = np.concatenate((Colours,Colours))
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=Colours)

### Import early human embryo scRNA-seq data and sample info

In [ ]:
### Load data
adata = ad.read_h5ad(path+"ESFS_Delile_2019.h5ad")
# adata = ad.read_h5ad(path+"Delile_2019.h5ad")
# adata = ad.read_h5ad(path+"DelileDF.h5ad")

### Create and save a scaled version of the scRNA-seq counts matrix
Create and save a scaled version of the scRNA-seq counts matrix where each genes has their observed expression values clipped if above a percentile threshold (default=97.5) and are then normalised by their maximum values so that all values are between 0-1, which is a requirment for the Entropy Sort Score (ESS) and Error Potential (EP) calcualtions.

In [ ]:
adata = ESFS.Create_Scaled_Matrix(adata,log_scale=False)

### Calculate ESS and EP matricies
Here we use the theory of Entropy Sorting to generate a pairwise gene correlation matrix (ESS matrix) and a correlation significance matrix (EP matrix). To speed up calculations parallel processing is implemented. To control the number of cores used for processing, vary the "Use_Cores" parameter which deaults to "Use_Cores=-1" which indicates the software will use n-1 cores, where n is the number of cores available on your machine. 
Please note that "Use_Cores=-1" is as special flag and "Use_Cores=-2" will not use n-2 cores. You should instead change "Use_Cores" to positive integer values of the number of cores you would like to use.

In [ ]:
adata = ESFS.Parallel_Calc_ES_Matricies(adata, Secondary_Features_Label="Self", save_matrices=np.array(["ESSs","EPs"]), Use_Cores=-1)


In [ ]:
# adata.write_h5ad(path+"Delile_2019.h5ad",compression='gzip')

In [ ]:
# adata = ad.read_h5ad(path+"Delile_2019.h5ad")

### Calculate gene importance rankings
We now calculate the gene importance weights and hence gene importance rankings for the remaining genes in the dataset. For further details see our manuscript.

In [ ]:
### ESS_Threshold is one of two paramaters that users should prioratise when trying to optimise the ESFS
# workflow. ESS_Threshold designates the upper threshold for edges between genes in the ESS matrix will be
# retained. For this earlky human embryo data, an ESS_Threshold of 0 works well and is a good starting point
# for most datasets. For other datasets, increasing the ESS_Threshold can be beneficial, as demonstrated in
# our Delile et al. 2019 neural tube scRNA-seq data example workflow.
ESS_Threshold = 0.01

### An array of known important genes for early human embryo development so that we may use them as a
# reference point for how genes are beign ranked/grouped. Replace these genes with those you are
# interested in in your data.
Known_Important_Genes = np.load(path+"Delile2019_Knowledge_Matrix_Genes.npy")
### Run the ES_Rank_Genes function while using "Exclude_Genes=Exclude_Batch_Effect_Genes" to exclude possible
# batch effect genes and "Known_Important_Genes=Known_Important_Genes" as reference point genes.
adata = ESFS.ES_Rank_Genes(adata,ESS_Threshold,Known_Important_Genes=Known_Important_Genes)

### Visualise clustering of top ranked genes in a UMAP generated from their pairwise ESS scores

In [ ]:
### Visualise top ranked genes graph
# Num_Top_Ranked_Genes is the second of the two paramaters that users should focus on when optimising
# the ESFS workflow for their data. Values between 3000-4000 are a typically good place to start.
Num_Top_Ranked_Genes = 3000

# The "Clustering" paramter can be set to an integer value if you wish to cluster genes in the UMAP with Kmeans clustering.
# Set "Clustering='hdbscan'" for automated density based clustering, and "Clustering=None" for no clustering.
Top_ESS_Genes, Gene_Clust_Labels, Gene_Embedding = ESFS.Plot_Top_Ranked_Genes_UMAP(adata,Num_Top_Ranked_Genes,Clustering="hdbscan",Known_Important_Genes=Known_Important_Genes)
plt.show()

### Visualise the cell clustering UMAPs for each of the groupings of genes identified in the previous step

This is stage of the worfklow is a critical step for trying out different input parameters to try and optimise the final results. The 2 main paramters to vary are the "Num_Top_Ranked_Genes" and the "Clustering" inputs for the Plot_Top_Ranked_Genes_UMAP function.

"Num_Top_Ranked_Genes" wil take the top ranked genes according to the cESFW algorithm. We recommend starting at around 3000-4000 genes and tweaking the "Clustering" paramater before trying higher or lower values of "Num_Top_Ranked_Genes".

The "Clustering" can either be an integer number, "hdbscan" or "None". An integer number will tell the algorithm to use Kmeans clustering with that number of clusters. "hdbscan" will use the hdbscan densitiy based clustering algorithm to automatically identify an optimal number of cluster according the gene UMAP embedding space.

In breif, you are seeking a gene cluster or combination of gene cluster that reveal biological structure of interest by excluding clusters of genes that contribute a large amount of noise to downstream analysis.

In [ ]:
Gene_Cluster_Embeddings, Gene_Cluster_Selected_Genes = ESFS.Get_Gene_Cluster_Cell_UMAPs(adata,Gene_Clust_Labels,Top_ESS_Genes,n_neighbors=50,min_dist=0.1,log_transformed=True)

ESFS.Plot_Gene_Cluster_Cell_UMAPs(adata,Gene_Cluster_Embeddings,Gene_Cluster_Selected_Genes,Cell_Label="domain")
plt.show()

### If you'd like to generate the cell UMAP for a specific cluster of a combination of clusters, use the specific_cluster parameter

For this workflow we find cluster 2 to be most informative of neural tube progenitor to neuron differentation and choose plot the resulting UMAP from these genes.

In [ ]:
Gene_Cluster_Embeddings, Gene_Cluster_Selected_Genes = ESFS.Get_Gene_Cluster_Cell_UMAPs(adata,Gene_Clust_Labels,Top_ESS_Genes,specific_cluster=[2],n_neighbors=40,min_dist=0.1,log_transformed=True)

ESFS.Plot_Gene_Cluster_Cell_UMAPs(adata,Gene_Cluster_Embeddings,Gene_Cluster_Selected_Genes,Cell_Label="domain")
plt.show()

As outlined in our mansucript, using this high resolution UMAP embedding from ESFS, we were able to perform unsupervised Leiden clustering and re-annotate mislabled cells from previous analyses.

In [ ]:
ESFS.Plot_Gene_Cluster_Cell_UMAPs(adata,Gene_Cluster_Embeddings,Gene_Cluster_Selected_Genes,Cell_Label="This_Paper_Reannoations")
plt.show()

### Finding marker genes
An important task in scRNA-seq analysis is to find marker genes for distinct populations. Commonly this is acheived by using statistical tests to perform differential expressed gene (DEG) analysis. However, as discussed in our manuscript, this process is limited by the requirement to identify a set of discrete non-overlapping clusters. Here we provide code to efficiently identify genes expression profiles enriched in the combinatorial cluster space of an intentionally over clustered dataset. This is possible because ES provides a mathermatically rigorous way to turn the intractable combinatorial cluster problem into a linearly complex problem, solvable in a minutes.

We start by taking the intentionally overclustered sample labels, Leiden_Clusts, and converting them into a one-hot representation. These one-hot encoded clusters will serve as "Secondary_Features" to calculate the Entropy Sort Score (ESS) and Sort Gain (SG) metrics when compared against every feature/gene in adata.  

In [ ]:
from scipy.sparse import csc_matrix

Leiden_Clusts = np.load("/Users/radleya/Desktop/Leiden_Clusts.npy",allow_pickle=True)
adata.obs["Leiden_Clusts"] = Leiden_Clusts
Fixed_Features = csc_matrix(pd.get_dummies(Leiden_Clusts).astype("f"))

Secondary_Features_Label = "Leiden_Clusts_Secondary_Features"
Leiden_Clusts_Secondary_Features = csc_matrix(pd.get_dummies(Leiden_Clusts).astype("f"))
adata.obsm[Secondary_Features_Label] = Leiden_Clusts_Secondary_Features


Now we use Parallel_Calc_ES_Matricies to calculate the required metrics for ES combinatorial marker gene identification (save_matrices=np.array(["ESSs","SGs"])). 
Secondary_Features_Label designates a prefix for saving the ESS and SG outputs.

In [ ]:
adata = ESFS.Parallel_Calc_ES_Matricies(adata, Secondary_Features_Label=Secondary_Features_Label, save_matrices=np.array(["ESSs","SGs"]), Use_Cores=-1)

With the ESS and SG matricies saved to adata, we can use the Find_Max_ESSs function to identify which combination of Leiden Clusters maximises the correlation (ESS) or each feature/gene in adata. The most important output of Find_Max_ESSs is a 2D array where each row is the samples of adata and each column is a feature representing the identified combination of Leiden Clusters that maximises the ESS score of the respective feature/gene column in adata.

In [ ]:
adata = ESFS.Find_Max_ESSs(adata,Secondary_Features_Label)

The 2D array of combined Leiden Clusters that maximise adata feature/gene ESSs is then used by Parallel_Calc_ES_Matricies as secondary features to obtain the ESS scores of each feature/gene in adata against each identified structure maximising feature. This creates a 2D pairwise adata feature/gene Vs coars grain combined Leiden Cluster matrix.

In [ ]:
Secondary_Features_Label = "Leiden_Clusts_Secondary_Features_Max_ESS_Features"
adata = ESFS.Parallel_Calc_ES_Matricies(adata, Secondary_Features_Label=Secondary_Features_Label, save_matrices=np.array(["ESSs"]), Use_Cores=-1)

Now that we have a coarse grain represenation of where gene structure is maximised in different regions of the data, we can use Find_Minimal_Combinatorial_Gene_Set to identify a minimal set of clusters/genes that captures that most unique/non-overlapping structure in the data. This minimal gene set may be thought of as an optimised set of unsuperised marker genes.

In [ ]:

Secondary_Features_Label
Num_Genes = 300
Chosen_Clusters, Chosen_Genes, Chosen_Pairwise_ESSs = ESFS.Find_Minimal_Combinatorial_Gene_Set(adata,Num_Genes,Secondary_Features_Label,Resolution=0.75,Num_Reheats=3)

# import seaborn as sns
# sns.clustermap(Chosen_Pairwise_ESSs)
# plt.savefig("/Users/radleya/The Francis Crick Dropbox/BriscoeJ/Radleya/ESFS_Paper/ESFS Figures/Paragi2020_Plots/Matched_Plots/" + "50_Factor_Heatmap" + ".png",dpi=600)
# plt.close()
# plt.show()

We can use knn_Smooth_Gene_Expression to get a knn mean smoothed representation of the data. We only use this for visualisation purposes.

In [ ]:
adata = ESFS.knn_Smooth_Gene_Expression(adata, Gene_Cluster_Selected_Genes[0], knn=30, metric='correlation', log_scale=False)

In [ ]:
ESFS.Plot_Gene_Cluster_Cell_UMAPs(adata,Gene_Cluster_Embeddings,Gene_Cluster_Selected_Genes,Cell_Label="Sox2") # Hypoblast marker

In [ ]:
import matplotlib.animation as animation
from IPython.display import Image, display
import tempfile
import os

def Display_Chosen_Genes_gif(Embedding,Chosen_Genes,adata):
    # Initialize figure and axis
    fig, ax = plt.subplots(figsize=(6, 6))

    # Function to update each frame
    def update(i):
        ax.clear()  # Clear previous frame
        Gene = Chosen_Genes[i]
        Exp = adata[:,Gene].layers["Smoothed_Expression"].A.ravel()
        Order = np.argsort(Exp)
        #
        ax.set_title(Gene, fontsize=22)
        Vmax = np.percentile(Exp, 99)
        if Vmax == 0:
            Vmax = np.max(Exp)
        scatter = ax.scatter(Embedding[Order, 0], Embedding[Order, 1], c=Exp[Order], s=2, vmax=Vmax, cmap="seismic")
        #
        ax.set_xticks([])
        ax.set_yticks([])
        fig.subplots_adjust(0.02, 0.02, 0.98, 0.9)

        return scatter,
    #
    # Create animation
    Num_Frames = Chosen_Genes.shape[0]
    ani = animation.FuncAnimation(fig, update, frames=Num_Frames, interval=500)
    #
    plt.close("all")
    #
    # Save animation as GIF
    with tempfile.TemporaryDirectory() as tmpdir:
        gif_path = os.path.join(tmpdir, "gene_expression.gif")
        ani.save(gif_path, writer=animation.PillowWriter(fps=0.5))
        # Display the GIF inline in Jupyter
        with open(gif_path, "rb") as f:
            display(Image(data=f.read(), format='png'))


In [ ]:
Embedding = Gene_Cluster_Embeddings[0]
Display_Chosen_Genes_gif(Embedding,Chosen_Genes,adata)

In [ ]:
# Combinatorial_Label_Info = adata.varm["Leiden_Clusts_Secondary_Features_Max_Combinatorial_ESSs"]
# Plot_Genes = Chosen_Genes[np.argsort(Combinatorial_Label_Info.loc[Chosen_Genes]["Max_ESSs"])].copy()
Cluster_ESSs = adata.varm['Leiden_Clusts_Secondary_Features_Max_ESS_Features_ESSs'][Chosen_Clusters,:]
col_max = np.max(Cluster_ESSs, axis=0, keepdims=True)

# Use broadcasting to create a mask where the max values are located
mask = Cluster_ESSs == col_max

# Apply the mask to keep only max values; all others become 0
Cluster_ESSs = Cluster_ESSs * mask